---
format:
  html:
    code-line-numbers: true
    code-overflow: wrap
    code-block-bg: true
    code-block-border-left: true
    highlight-style: Arrow
---

# The Bin Packing Problem {#sec-bin-packing}

The bin packing problem (BPP) is a classic optimization problem in computer science that involves packing a set of items into a minimum number of bins or containers, subject to certain constraints. The problem is also known as the container loading problem, or the bin packing optimization problem.

In the bin packing problem, we are given a set of items, each with a weight or size, and a set of bins, each with a fixed capacity. The goal is to pack the items into the bins such that the number of bins used is minimized.

The problem is NP-hard, which means that there is no known algorithm that can solve it in polynomial time. However, several heuristic algorithms have been developed that can provide good solutions in practice.

The bin packing problem has many real-world applications, such as optimizing the use of storage space in warehouses, minimizing the number of trucks needed for transportation, and optimizing the use of computer memory. It is also a fundamental problem in the study of computational complexity theory and has been extensively studied in the fields of operations research, computer science, and mathematics.

To formally express the bin packing problem (BPP), we use the notation of a set of items that need to be packed into a set of bins. The set of items is denoted as $\mathcal{I}$, while the set of bins is denoted as $\mathcal{J}$. Each item $i$ in $\mathcal{I}$ has a size denoted as $s_i$, and each bin $j$ in $\mathcal{J}$ has a fixed capacity denoted as $Q$. 
To solve the problem, we introduce two decision variables: $y_j$ and $x_{ij}$. 

- The variable $y_j$ is a binary variable that takes the value 1 if the bin $j$ is used and 0 otherwise. 
- The variable $x_{ij}$ is a binary variable that takes the value 1 if item $i$ is placed into bin $j$ and 0 otherwise. 

Using these decision variables, the BPP can be formulated as follows.

\begin{align}
    \text{min.} &\quad \sum_{j \in \mathcal{J}} y_j \label{bin-obj} \\
    \text{s.t.} &\quad \sum_{i \in \mathcal{I}} s_i x_{ij} \leq Q y_j, \ \forall j \in \mathcal{J} \label{bin-cons1} \\
    &\quad \sum_{j \in \mathcal{J}} x_{ij} = 1, \ \forall i \in \mathcal{I} \label{bin-cons2} \\
    &\quad y_j \in \{0, 1\}, \ \forall j \in \mathcal{J} \label{bin-cons3} \\
    &\quad x_{ij} \in \{0, 1\}, \ \forall i \in \mathcal{I}, j \in \mathcal{J} \label{bin-cons4}
\end{align}

In [10]:
class BppDataCenter:
    
    def __init__(self, num_bins, num_items, bin_capacity, item_sizes):
        self._num_bins = num_bins
        self._num_items = num_items
        self._bin_capacity = bin_capacity
        self._item_sizes = item_sizes
        
    @property
    def num_bins(self): return self._num_bins
    
    @property
    def num_items(self): return self._num_items
    
    @property
    def bin_capacity(self): return self._bin_capacity
    
    @property
    def item_sizes(self): return self._item_sizes

In [18]:
import numpy as np
from itertools import product
from ortools.linear_solver import pywraplp

class BppSolver:
    
    def __init__(self, data_center):
        self._data_center = data_center
        
        self._solver = None
        self._var_y = None
        self._var_x = None
        
        self._opt_obj = None
        self._opt_y = None
        self._opt_x = None
        
    def build_model(self):
        self._solver = pywraplp.Solver.CreateSolver('SCIP')
        self._create_variables()
        self._create_objective()
        self._create_constraints()
        
    def optimize(self):
        num_bins = self._data_center.num_bins
        num_items = self._data_center.num_items
        
        # model = self._solver.ExportModelAsLpFormat(False)
        # print(model)
        
        status = self._solver.Solve()
        if status == pywraplp.Solver.OPTIMAL:
            self._opt_obj = self._solver.Objective().Value()
            self._opt_y = [
                self._var_y[b].solution_value()
                for b in range(num_bins)
            ]
            self._opt_x = np.zeros((num_items, num_bins))
            for i, b in product(range(num_items),
                                range(num_bins)):
                self._opt_x[i][b] = \
                    self._var_x[i][b].solution_value()
        
    def _create_variables(self):
        num_bins = self._data_center.num_bins
        self._var_y = [
            self._solver.BoolVar(name=f'y_{b}')
            for b in range(num_bins)
        ]
        num_items = self._data_center.num_items
        self._var_x = np.empty((num_items, num_bins),
                            dtype=object)
        for i, b in product(range(num_items),
                            range(num_bins)):
            self._var_x[i][b] = \
                self._solver.BoolVar(name=f'x_{i, b}')
        
    def _create_objective(self):
        self._solver.Minimize(
            self._solver.Sum(
                self._var_y
            )
        )
        
    def _create_constraints(self):
        num_bins = self._data_center.num_bins
        num_items = self._data_center.num_items
        bin_capacity = self._data_center.bin_capacity
        item_sizes = self._data_center.item_sizes
        for b in range(num_bins):
            expr = [
                item_sizes[i] *
                self._var_x[i][b]
                for i in range(num_items)
            ]
            self._solver.Add(
                self._solver.Sum(expr) <= \
                    bin_capacity * self._var_y[b]
            )
            
        for i in range(num_items):
            expr = [
                self._var_x[i][b]
                for b in range(num_bins)
            ]
            self._solver.Add(
                self._solver.Sum(expr) == 1
            )
        

In [23]:
import os
import re

data_file = "./data/bpp/binpack1.txt"

with open(data_file) as f:
    num_instanes = int(f.readline())
    print(num_instanes)
    
    for inst in range(num_instanes):
        inst_name = f.readline()
        print(inst_name)
        bin_capacity, num_items, min_num_bins = f.readline().split()
        print(bin_capacity)
        print(num_items)
        print(min_num_bins)
        sizes = []
        for i in range(int(num_items)):
            line = f.readline()
            number = int(re.findall(r'\d+', line)[0])
            sizes.append(number)
        
        data_center = BppDataCenter(num_bins=int(min_num_bins),
                                    num_items=15,
                                    bin_capacity=int(bin_capacity),
                                    item_sizes=sizes)

        solver = BppSolver(data_center)                       
        solver.build_model()
        solver.optimize()
        print(f'obj = {solver._opt_obj}')
        break

20
 u120_00 

150
120
48
\ Generated by MPModelProtoExporter
\   Name             : 
\   Format           : Free
\   Constraints      : 63
\   Variables        : 768
\     Binary         : 768
\     Integer        : 0
\     Continuous     : 0
Minimize
 Obj: +1 y_0 +1 y_1 +1 y_2 +1 y_3 +1 y_4 +1 y_5 +1 y_6 +1 y_7 +1 y_8 +1 y_9 +1 y_10 +1 y_11 +1 y_12 +1 y_13 +1 y_14 +1 y_15 +1 y_16 +1 y_17 +1 y_18 +1 y_19 +1 y_20 +1 y_21 +1 y_22 +1 y_23 +1 y_24 +1 y_25 +1 y_26 +1 y_27 +1 y_28 +1 y_29 +1 y_30 +1 y_31 +1 y_32 +1 y_33 +1 y_34 +1 y_35 +1 y_36 +1 y_37 +1 y_38 +1 y_39 +1 y_40 +1 y_41 +1 y_42 +1 y_43 +1 y_44 +1 y_45 +1 y_46 +1 y_47 
Subject to
 auto_c_000000000: -150 y_0 +42 x_(0,_0) +69 x_(1,_0) +67 x_(2,_0) +57 x_(3,_0) +93 x_(4,_0) +90 x_(5,_0) +38 x_(6,_0) +36 x_(7,_0) +45 x_(8,_0) +42 x_(9,_0) +33 x_(10,_0) +79 x_(11,_0) +27 x_(12,_0) +57 x_(13,_0) +44 x_(14,_0)  <= 0
 auto_c_000000001: -150 y_1 +42 x_(0,_1) +69 x_(1,_1) +67 x_(2,_1) +57 x_(3,_1) +93 x_(4,_1) +90 x_(5,_1) +38 x_(6,_1) +36